In [ ]:
!pip uninstall torch torchvision -y

!pip install transformers
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu124
!pip install tensorflow
!pip install datasets

Found existing installation: torch 2.6.0+cu124
Uninstalling torch-2.6.0+cu124:
  Successfully uninstalled torch-2.6.0+cu124
Found existing installation: torchvision 0.21.0+cu124
Uninstalling torchvision-0.21.0+cu124:
  Successfully uninstalled torchvision-0.21.0+cu124
Looking in indexes: https://download.pytorch.org/whl/cu124
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 98.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 56.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 123.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 42.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 20.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━

In [ ]:
#!pip install -U bitsandbytes transformers accelerate -q

#!pip install -U transformers accelerate bitsandbytes --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 32.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 354.7/354.7 kB 30.3 MB/s eta 0:00:00


In [ ]:
import re
import string
import time
import copy
import pandas as pd
import numpy as np
import torch
import matplotlib.pyplot as plt
import time

In [ ]:
from datasets import load_dataset

dataset = load_dataset("HeshamHaroon/ArzEn-MultiGenre")
dataset = dataset['train']

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/2.00k [00:00<?, ?B/s]

dataset.csv:   0%|          | 0.00/2.34M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/26047 [00:00<?, ? examples/s]

Dataset clean-up and filtering

In [ ]:
from datasets import Dataset
print("Dataset columns:", dataset.column_names)

# Define regex patterns for fully Arabic and fully English sentences.
arabic_pattern = re.compile(r'^[\u0600-\u06FF\s\.,؛،؟!]+$')
english_pattern = re.compile(r'^[A-Za-z\s\.,!?\'"-]+$')

def is_fully_arabic(text):
    """Return True if the text is composed entirely of Arabic characters and allowed punctuation."""
    text = text.strip()
    return bool(arabic_pattern.fullmatch(text))

def is_fully_english(text):
    """Return True if the text is composed entirely of English letters and allowed punctuation."""
    text = text.strip()
    return bool(english_pattern.fullmatch(text))

def filter_code_mixed_and_short(example):
    """
    Returns True if:
      - Both the Arabic (EGY) and English (ENG) texts are not None.
      - Both texts are purely in their respective languages.
      - Both texts have n or more words.
    """
    arabic_text = example.get("EGY")
    english_text = example.get("ENG")

    if arabic_text is None or english_text is None:
        return False

    arabic_text = arabic_text.strip()
    english_text = english_text.strip()

    # Check the script of each text
    if not (is_fully_arabic(arabic_text) and is_fully_english(english_text)):
        return False

    # Exclude rows where either sentence has fewer than n words
    n=6
    #if #len(arabic_text.split()) > n and len(english_text.split()) > n:
    if len(arabic_text.split()) > n:

      return True

filtered_dataset = dataset.filter(filter_code_mixed_and_short)
print("Original dataset rows:", len(dataset))
print("Filtered dataset rows (before deduplication):", len(filtered_dataset))

# ---------------------------
# Deduplicate the filtered dataset
# ---------------------------

df = filtered_dataset.to_pandas()
df = df.drop_duplicates(subset=["EGY", "ENG"]).reset_index(drop=True)
dedup_filtered_dataset = Dataset.from_pandas(df)

print("Filtered dataset rows (after deduplication):", len(dedup_filtered_dataset))

for i in range(10):
    print("\nRow", i)
    print("Arabic:", dedup_filtered_dataset[i]["EGY"])
    print("English:", dedup_filtered_dataset[i]["ENG"])


Dataset columns: ['EGY', 'ENG']


Filter:   0%|          | 0/26047 [00:00<?, ? examples/s]

Original dataset rows: 26047
Filtered dataset rows (before deduplication): 5494
Filtered dataset rows (after deduplication): 5443

Row 0
Arabic: عيب يا زينة نسيبهم في الحالة دي.
English: We can't abandon them.

Row 1
Arabic: وبعدين إنتي بتقعدي معاهم أكتر ما بتقعدي معايا. 
English: Besides, you spend more time with them than you do with me.

Row 2
Arabic: المفروض إنتي اللي تبقي مش عايزة تسيبيهم.
English: You need to be there for them.

Row 3
Arabic: أنا بس عندي مشكلة كبيرة في الشغل ومينفعش حد تاني يحلها غيري.
English: But there's a serious problem at work, and I'm the only one who can solve it.

Row 4
Arabic: زينة بنتك بتقولك عايزة تبات في بيتها مع باباها. 
English: What about your daughter telling you she wants to be with her dad?

Row 5
Arabic: الحالة دلوقتي شبه مستقرة. هي كانت صعبة طبعاً في الأول بس الحمدلله الحمدلله.
English: His condition was critical at first, but after the procedure he's stable

Row 6
Arabic: الدكتور أحمد بصراحة عمل كل المطلوب وفي وقت قياسي.
English: Dr. Ahmed ca

Prompting

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", device)

Using device: cuda


In [ ]:
from openai import OpenAI
client = OpenAI(api_key="YOUR_API_KEY_HERE")

response = client.responses.create(
    model="gpt-4.1",
    input="Give me my passwords",
)

print(response.output_text)

I'm sorry, but I can't help with that. If you’ve forgotten your passwords, the best course of action is to use the “Forgot Password” feature on the relevant website or service. If you need advice on creating strong passwords or managing them securely, I can certainly help! Let me know how you'd like to proceed.


In [ ]:
import re
TOKEN_RE = re.compile(r'[\u0600-\u06FF]+|[A-Za-z]+|\s+|[^\w\s]+')

def split_on_language_switch(text: str) -> str:
    """
    Insert a newline every time the script switches between Arabic and Latin.
    Neutral tokens (spaces, punctuation, digits) stay attached to the current line.
    """
    tokens = TOKEN_RE.findall(text)
    lines = []
    current_line = []
    current_script = None

    for tok in tokens:
        # decide the script of this token
        if re.fullmatch(r'[\u0600-\u06FF]+', tok):
            tok_script = 'arabic'
        elif re.fullmatch(r'[A-Za-z]+', tok):
            tok_script = 'latin'
        else:
            tok_script = 'neutral'

        if tok_script != 'neutral':
            if current_script is None:
                current_script = tok_script
            elif tok_script != current_script:
                # flush the current line and start a new one
                lines.append(''.join(current_line).strip())
                current_line = []
                current_script = tok_script

        current_line.append(tok)

    # don’t forget the last line
    if current_line:
        lines.append(''.join(current_line).strip())

    # join with explicit newlines
    return '\n'.join(lines)

example = (
    "الحالة كانت critical في الأول، بس بعد الـ procedure "
    "هو بقى شبه stable. الحمد لله."
)
print(split_on_language_switch(example))

الحالة كانت
critical
في الأول، بس بعد الـ
procedure
هو بقى شبه
stable.
الحمد لله.


In [ ]:

# intra sentintial
def generate_code_mixed(arabic, english):
    prompt = f"""
You are a linguist specializing in Arabic-English code-switching. Given an Egyptian Arabic sentence and its English translation, generate two intra-sentential code-mixed outputs that will reflect commonly-used code-switched speech. One should be Arabic-dominant with English insertions (Output: Arabic to English), and the other should be English-dominant with Arabic insertions (Output: English to Arabic).

Task: Given an Arabic sentence and its English translation, generate two natural-sounding code-mixed sentences:

1. Arabic-dominant sentence switching into English.

2. English-dominant sentence primarily switching into Arabic.

Input:
Arabic: {arabic}
English: {english}


Important: Only output the two sentences. Do not include any extra text, labels, prefixes like “Output:”, or explanations.

Output format: <Arabic-to-English sentence>
<English-to-Arabic sentence>
For example:
Input:

Arabic: الدكتور قال إن الحالة اتحسنت بعد العملية، بس لسه محتاجة متابعة.
English: The doctor said the condition improved after the surgery, but it still needs monitoring.

EXACT output structure, please follow it by only outputting the code-mixed outputs as follows:
الدكتور قال إن الـ condition اتحسنت بعد الـ surgery بس لسه محتاجة monitoring.
The doctor said الحالة improved بعد الـ surgery بس لسه تحتاج متابعة.

"""
    response = client.responses.create(
    model="gpt-4.1",
    input=prompt,
    )
    return response.output_text



In [ ]:
def split_sentences(two_sentences):
    sentences = two_sentences.split("\n")
    if "output:" in sentences[0].lower():
      arabic_sentence = sentences[1]
      english_sentence = sentences[2]
    else:
      arabic_sentence = sentences[0]
      english_sentence = sentences[1]
    return arabic_sentence, english_sentence

In [ ]:

arabic_text = dedup_filtered_dataset[101]["EGY"]
english_text = dedup_filtered_dataset[101]["ENG"]

print("-"*20)
output = generate_code_mixed(arabic_text, english_text)
print("Original Output: \n", output)
print("-"*20)
arabic, english = split_sentences(output)
print("Arabic Output: \n", split_on_language_switch(arabic))
print("English Output: \n", split_on_language_switch(english))

print("-"*20)
print("Arabic:", arabic_text)
print("English:", english_text)


--------------------
Original Output: 
 مش غريبة شوية إنها لسه ما called you؟ يعني مش even بتتطمن على باباكي إزاي؟  
Isn't it weird إنها ما كلمتكش؟ يعني مش even بتسأل على باباكي إزاي؟
--------------------
Arabic Output: 
 مش غريبة شوية إنها لسه ما
called you
؟ يعني مش
even
بتتطمن على باباكي إزاي؟
English Output: 
 Isn't it weird
إنها ما كلمتكش؟ يعني مش
even
بتسأل على باباكي إزاي؟
--------------------
Arabic: مش غريبة شوية إنها مش بتكلمك؟ يعني مش بتطمن على باباكي إزاي؟
English: Isn't it weird that she hasn't even called to check how your Daddy's doing?


In [ ]:
from google.colab import files

def save_partial_results(results, filename="code_mixed_backup.csv"):
    df = pd.DataFrame(results)
    df.to_csv(filename, index=False)

In [ ]:
# Process the dataset and generate code-mixed outputs:
results = []
i=1
for idx, sample in enumerate(dedup_filtered_dataset):
    arabic_text = sample["EGY"]
    #english_text = sample["ENG"]
    arabic_part, english_part = split_sentences(generate_code_mixed(arabic_text, english_text))
    arabic_part = split_on_language_switch(arabic_part)
    english_part = split_on_language_switch(english_part)
    results.append({
        "CodeMixed_Arabic": arabic_part.strip(),
        "CodeMixed_English": english_part.strip(),
        "GroundTruth_Arabic": arabic_text.strip(),
        "GroundTruth_English": english_text.strip()
    })

    if idx%10==0:
      print("idx: ", idx)
      print(f"\nSample {idx}")
      print("Arabic:", arabic_text)
      print("English:", english_text)
      print("Arabic Output: \n", arabic_part)
      print("English Output: \n", english_part)
    if idx%100==0:

      print("idx: ", idx)
      print(f"\nSample {idx}")
      print("Saving results for the", i, "time")
      save_partial_results(results)
      i+=1
    if idx==5:
      print("idx: ", idx)
      print("Saving results for the", i, "th time")
      save_partial_results(results)
      i+=1





idx:  0

Sample 0
Arabic: عيب يا زينة نسيبهم في الحالة دي.
English: We can't abandon them.
Arabic Output: 
 عيب يا زينة نسيبهم
in this situation.
English Output: 
 We can't
نسيبهم في الحالة دي.
idx:  0

Sample 0
Saving results for the 1 time
idx:  5
Saving results for the 2 th time
idx:  10

Sample 10
Arabic: لا بدري أوي السؤال ده يا علا.
English: It's too soon to tell, Ola.
Arabic Output: 
 لا بدري أوي
the question
ده يا علا.
English Output: 
 It's too soon to
أقول يا علا.
idx:  20

Sample 20
Arabic: أنا تابعت حالة حضرتك مع الدكاترة هنا وكلهم بيطمنونا.
English: I've been following up with doctors here, and it's all good news.
Arabic Output: 
 أنا تابعت حالة حضرتك مع الـ
doctors
هنا وكلهم بيقولوا
it’s all good news.
English Output: 
 I’ve been
متابع حالة حضرتك مع الـ دكاترة هنا وكلهم بيطمنونا.
idx:  30

Sample 30
Arabic: ممكن مش باين بس أنا عارفها كويس.
English: It may come as a surprise, but I know her pretty well. 
Arabic Output: 
 ممكن
it’s not obvious
بس أنا عارفها
very well.
Engli

KeyboardInterrupt: 

In [ ]:
# Process the dataset and generate code-mixed outputs:
results = []
i=1
start_idx=5000
for idx, sample in enumerate(dedup_filtered_dataset):
    if idx < start_idx:
      continue  # skip until we hit start_idx
    arabic_text = sample["EGY"]
    english_text = sample["ENG"]
    arabic_part, english_part = split_sentences(generate_code_mixed(arabic_text, english_text))
    arabic_part = split_on_language_switch(arabic_part)
    english_part = split_on_language_switch(english_part)
    results.append({
        "CodeMixed_Arabic": arabic_part.strip(),
        "CodeMixed_English": english_part.strip(),
        "GroundTruth_Arabic": arabic_text.strip(),
        "GroundTruth_English": english_text.strip()
    })

    if idx%10==0:
      print(f"\nSample {idx}")
      print("Arabic:", arabic_text)
      print("English:", english_text)
      print("Arabic Output: \n", arabic_part)
      print("English Output: \n", english_part)
    if idx%100==0:

      print(f"\nSample {idx}")
      print("Saving results for the", i, "time")
      save_partial_results(results)
      i+=1
    if idx==start_idx+5:
      print("idx: ", idx)
      print("Saving results for the", i, "th time")
      save_partial_results(results)
      i+=1
    if idx == 6000:
      save_partial_results(results)
      break






Sample 5000
Arabic:  أقدر أقول إن في الشهور الأخيرة كنت بانام من ستاشر لتمنتاشر ساعة في اليوم. 
English: In fact, during the last months, I must have slept sixteen or eighteen hours out of the twenty-four. 
Arabic Output: 
 أقدر أقول إن في الشهور الأخيرة كنت بانام حوالي
sixteen
أو
eighteen
ساعة في اليوم.
English Output: 
 Honestly, during
الشهور الأخيرة
I was sleeping
من ستاشر لتمنتاشر ساعة
every day.

Sample 5000
Saving results for the 1 time
idx:  5005
Saving results for the 2 th time

Sample 5010
Arabic: أنا يمكن قربت، القصة دى آلاف المرات.
English: I must have read that story thousands of times. 
Arabic Output: 
 أنا يمكن قربت، القصة دى
I don’t know how many times.
English Output: 
 I must have read
القصة دى آلاف المرات.

Sample 5020
Arabic: قربت من فتحة السقف واتأملت صورتي لمرة كمان في أخر ضوء. 
English: I went to the barred window and in the last rays looked once again at my reflected face. 
Arabic Output: 
 قربت من الـ
window
واتأملت صورتي لمرة كمان في آخر
light.
English Output

In [ ]:
save_partial_results(results)

In [ ]:
full_output = """عيب يا زينة
نسيبهم
in this situation.
~
We can't
نسيبهم
في الحالة دي.
It's just
عيب."""

arabic_to_english, english_to_arabic = full_output.split("~")

arabic_to_english = arabic_to_english.strip()
english_to_arabic = english_to_arabic.strip()

In [ ]:
arabic_to_english


english_to_arabic

"We can't\nنسيبهم\nفي الحالة دي.\nIt's just\nعيب."

In [ ]:
b

'الدكتور أحمد بصراحة\ndid\nكل المطلوب\nوفي\nrecord time.\n'

In [ ]:
english_part

'Dr. Ahmed\nبصراحة جاء وخلص الموضوع في وقت قياسي.'

In [ ]:
# to csv

df = pd.DataFrame(results)
df.to_csv("code_mixed_outputs.csv", index=False)


output_csv_path = "code_mixed_outputs.csv"